<a href="https://colab.research.google.com/github/M-H-Amini/MachineLearning-AUT/blob/master/MLe_Lec2_LogisticRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# In The Name Of ALLAH
# Machine Learning *elementary* Course
## Amirkabir University of Technology
### Mohammad Hossein Amini (mhamini@aut.ac.ir)
# Lecture 2 - Logistic Regression

<img src="https://drive.google.com/uc?id=144SDpgv7EEy6Og1ZFNIv_nBaugKGiSCE" width="400">



# Introduction

The theoretical stuff has been discussed in the video lectures. Let's implement a little...

First of all, we should import some modules.

In [0]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
print(tf.__version__)
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Discovering Sigmoid
Let's see **sigmoid** function in action.

In [0]:
def sigmoid(x):
  return 1/(1 + np.exp(-x))


x = np.linspace(-10, 10, 100)
plt.figure()
plt.plot(x, sigmoid(x), 'r')
plt.title(r'$\sigma (x) = \frac{1}{1+e^{-x}} $', fontsize = 30)
plt.show()

<img src="https://drive.google.com/uc?id=1SdMUDfOhtOsfDMiDVsE4InR5fVSOAvF2" width="300">


# Logistic Regression (Hand Coding!)
Let's implement some simple examples just using **numpy**!

First of all, let's create a simple dataset.

In [0]:
x = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y = np.array([[0], [0], [0], [1]])
print(x)
print(y)

A simple function to show the situation can be so helpful.

In [0]:
def show(x, y, w=None):
  label = {0: 'ro', 1: 'bx'}
  plt.figure()
  for i in range(y.shape[0]):
    plt.plot(x[i, 0], x[i, 1], label[y[i, 0]])
  if w is not None:
    p1 = (0, -w[0, 0]/w[2, 0])
    p2 = (-w[0, 0]/w[1, 0], 0)
    plt.plot([p1[0], p2[0]], [p1[1], p2[1]], 'g')
  plt.show()

w = np.array([[-0.5], [0.1], [1]])
show(x, y, w)

Just like the **Linear Regression** Notebook, we implement functions **h** and **train_step**.

In [0]:
alpha = 0.1
def h(x, w, has_bias = True):
  if not has_bias:
    x = np.concatenate((np.ones((x.shape[0], 1)), x), axis=1)
  return sigmoid(np.dot(x, w))
def train_step(x, y, w):
  x = np.concatenate((np.ones((x.shape[0], 1)), x), axis=1)
  w = w + alpha*np.dot(x.T , (y-h(x, w)))
  return w

print('Before: ', w.T)
w = train_step(x[0:1, :], y[0:1, :], w)
print('After: ', w.T)

In [0]:
#index = np.random.randint(0, y.shape[0])
alpha = 0.1
index = 3
w = train_step(x[index:index+1, :], y[index:index+1, :], w)
show(x, y, w)

In [0]:
alpha = 0.03
for i in range(100):
  index = np.random.randint(0, y.shape[0])
  w = train_step(x[index:index+1, :], y[index:index+1, :], w)

show(x, y, w)

# Importing Dataset
Today, we'll be using **Heart Disease UCI** dataset. It predicts the likelihood of a patient having heart disease. Let's import it in our program.

In [0]:
ds = pd.read_csv('heart.csv')
ds.head()

In [0]:
ds = pd.read_csv('heart.csv')
ds = ds.sample(frac=1).reset_index(drop=True)
a = pd.get_dummies(ds['cp'], prefix='cp')
b = pd.get_dummies(ds['slope'], 'slope')
c = pd.get_dummies(ds['thal'], 'thal')
ds = pd.concat([ds, a, b, c], axis=1)
ds = ds.drop(columns=['cp', 'slope', 'thal'])
y = np.array(ds['target'])
y = y[:, np.newaxis]
ds = ds.drop(columns=['target'])
x = np.array(ds)
#maximum = np.array(ds.max())
#ds = ds/maximum
ds.head()

In [0]:
split = 0.8
no_of_trains = int(ds.shape[0]*split)
X_train = x[:no_of_trains, :]
Y_train = y[:no_of_trains, :]
X_test = x[no_of_trains:, :]
Y_test = y[no_of_trains:, :]

# Tensorflow Codes
Our implementation in tensorflow is much like the previous session.

In [0]:
X = tf.constant(X_train, dtype=tf.float32)
Y = tf.constant(Y_train, dtype=tf.float32)

w = tf.Variable(np.random.randn(X.shape[1]+1, 1), dtype=tf.float32)

In [0]:
def h(x, w):
  x = tf.concat((tf.ones((x.shape[0], 1)), x), axis=1)
  return tf.sigmoid(tf.matmul(x,w))

def cost(x, y, w):
  return tf.reduce_mean(tf.losses.binary_crossentropy(y, h(x, w)))

In [0]:
optimizer = tf.optimizers.Adam()
#w = tf.Variable(np.random.randn(X.shape[1]+1, 1)/1000, dtype=tf.float32)
w = tf.Variable(np.zeros((X.shape[1]+1, 1)), dtype=tf.float32)

def train_step(x, y, w, verbose=0):
  with tf.GradientTape() as t:
    J = cost(x, y, w)
  if verbose:
    print('Loss: {}'.format(J))
  w_grads = t.gradient(J, w)
  optimizer.apply_gradients(zip([w_grads], [w]))
  return w

print('Before: ', w)
w = train_step(X[index:index+1, :], Y[index:index+1, :], w, verbose = 1)
print('After: ', w)
print(cost(X, Y, w))

<img src="https://drive.google.com/uc?id=17X8qdM19JAlv53hlNJSolrRjXYfaOel-" width="300">


In [0]:
def train(X, Y, max_iters=1000, min_cost=0.01, w=None, verbose=0):
  if w is None:
    w = tf.Variable(np.random.randn(X.shape[1]+1, 1), dtype=tf.float32)
  for i in range(max_iters):
    index = np.random.randint(0, X.shape[0])
    w = train_step(X[index:index+1, :], Y[index:index+1, :], w)
    if verbose:
      print('Cost: ', cost(X, Y, w).numpy())
  print("Training Done...")
  print("Cost: {}".format(cost(X, Y, w).numpy()))
  print("w = ", w)
  return w

w = train(X, Y, w=w, max_iters=1000, verbose=0)

In [0]:
predictions = h(X_test, w)
for i in range(X_test.shape[0]):
  print('No {:2}, Target: {}, Output: {:.2f}'.format(i+1, Y_test[i,0], predictions.numpy()[i,0]))

In [0]:
corrects = 0
wrongs = 0
for i in range(Y_test.shape[0]):
  if abs(Y_test[i, 0] - predictions.numpy()[i, 0])<=0.5:
    corrects += 1
  else:
    wrongs += 1

print(f'Corrects: {corrects}, Wrongs: {wrongs}')
print('Accuracy: {}'.format(corrects/(corrects + wrongs)))
